# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 4667, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 4667 (delta 138), reused 110 (delta 42), pack-reused 4388 (from 1)
Receiving objects: 100% (4667/4667), 176.44 MiB | 21.06 MiB/s, done.
Resolving deltas: 100% (2719/2719), done.
Updating files: 100% (413/413), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 24.1 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprecated NumPy API, disabl

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithoutKFCV/MatrixFactorization'

np.random.seed(42)

## Import the repository

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'NMF',
    'metric': 'MAP',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_NMF_MAP.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [12]:
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

def objective_function_NMF(optuna_trial):
    
    recommender_instance = NMFRecommender(URM_train)

    full_hyperp = {
                    "num_factors": optuna_trial.suggest_int("num_factors", 10, 300),
                    "l1_ratio": optuna_trial.suggest_float("l1_ratio", 0, 1),
                    "init_type": optuna_trial.suggest_categorical("init_type", ["random", "nndsvda"]),
                    "solver_beta_loss": optuna_trial.suggest_categorical("solver_beta_loss", [
                                                                    "coordinate_descent:frobenius", 
                                                                    "multiplicative_update:frobenius", 
                                                                    "multiplicative_update:kullback-leibler"
                                                                ]) 
                }
       
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_NMF, n_trials=10)

[I 2024-12-26 10:40:23,365] Using an existing study with name 'hyperparameters_tuning_NMF_MAP' instead of creating a new one.


NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 52.98 min
EvaluatorHoldout: Processed 35595 (100.0%) in 43.53 sec. Users per second: 818


[I 2024-12-26 11:34:05,800] Trial 46 finished with value: 0.02216209138522634 and parameters: {'num_factors': 276, 'l1_ratio': 0.7111769929220721, 'init_type': 'random', 'solver_beta_loss': 'coordinate_descent:frobenius'}. Best is trial 31 with value: 0.02245555154215051.


NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 1.52 hour
EvaluatorHoldout: Processed 35595 (100.0%) in 45.07 sec. Users per second: 790


[I 2024-12-26 13:06:04,124] Trial 47 finished with value: 0.022584957090013735 and parameters: {'num_factors': 285, 'l1_ratio': 0.7187695934990422, 'init_type': 'random', 'solver_beta_loss': 'coordinate_descent:frobenius'}. Best is trial 47 with value: 0.022584957090013735.


...

[I 2024-12-26 17:42:46,253] Trial 53 finished with value: 0.022728342664499034 and parameters: {'num_factors': 298, 'l1_ratio': 0.978384022296767, 'init_type': 'nndsvda', 'solver_beta_loss': 'coordinate_descent:frobenius'}. Best is trial 51 with value: 0.0232130727741766.


NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 31.74 min
EvaluatorHoldout: Processed 35595 (100.0%) in 44.11 sec. Users per second: 807


[I 2024-12-26 18:15:15,164] Trial 54 finished with value: 0.02292241196708092 and parameters: {'num_factors': 299, 'l1_ratio': 0.9713955208790358, 'init_type': 'nndsvda', 'solver_beta_loss': 'coordinate_descent:frobenius'}. Best is trial 51 with value: 0.0232130727741766.


NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 32.65 min
EvaluatorHoldout: Processed 35595 (100.0%) in 43.66 sec. Users per second: 815


[I 2024-12-26 18:48:37,949] Trial 55 finished with value: 0.02295057937957189 and parameters: {'num_factors': 300, 'l1_ratio': 0.9902468735316549, 'init_type': 'nndsvda', 'solver_beta_loss': 'coordinate_descent:frobenius'}. Best is trial 51 with value: 0.0232130727741766.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = NMFRecommender(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 32.06 min


# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_NMF_MAP.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithoutKFCV/MatrixFactorization/NMFRecommender/OptimizingMAP/best_params_NMF_MAP.json' created successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithoutKFCV/MatrixFactorization/NMFRecommender/OptimizingMAP/history_NMF_MAP.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithoutKFCV/MatrixFactorization/NMFRecommender/OptimizingMAP/Submission/submission_NMF_MAP.csv' created successfully.
